# Vision Transformers in Action
## Building and Training ViT to classify images


### Learning Objectives
- Understand how Vision Transformers process images as sequences
- Implement ViT from scratch using PyTorch
- Train on real datasets from Hugging Face

In [ ]:
try: 
    import google.colab
    !pip install transformers datasets torch torchvision matplotlib numpy pillow tqdm
    ON_COLAB = True
except:
    print("Running Locally")
    ON_COLAB = False

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.utils import make_grid

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import math
from tqdm.auto import tqdm
from datasets import load_dataset
import random

# Set device and seeds for reproducibility
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(42)
np.random.seed(42)
print(f"Using device: {device}")

## Part 1: Loading and Preparing Data

### 1.1 First let's download the `food101' Dataset from Hugging Face

If running locally, you can save time downloading the dataset by downloading from Google Drive via [this link](https://drive.google.com/file/d/1W_0BAmoT2gkSKf_vEiEJYUGLX8fKOBK_/view?usp=sharing) with Monash account. Unzip it and paste that to the `datasets` folder.

In [ ]:
dataset_name = "food101"
def prepare_data(dataset_name=dataset_name):
    """Load dataset from Hugging Face and create data loaders"""

    # Load dataset
    train_dataset = load_dataset(dataset_name, split="train[:5000]")
    val_dataset = load_dataset(dataset_name, split="validation[:500]")

    dataset = load_dataset(dataset_name, split="train")
    dataset = dataset.shuffle(seed=42) # Shuffle for randomness
    dataset_small = dataset.select(range(5000)) # Select only 5000 samples
    splits = dataset_small.train_test_split(test_size=0.1, seed=42)     # Split into 90/10
    train_dataset = splits["train"]
    val_dataset = splits["test"]

    # class names
    class_names = dataset_small.features["label"].names

    train_transform = transforms.Compose([
        # Can add more data augmentation here
        # Suggestions: RandomHorizontalFlip, RandomCrop, ColorJitter
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    return train_dataset, val_dataset, train_transform, val_transform, class_names

class ImageDataset(Dataset):
    """Custom dataset wrapper for the dataset"""

    def __init__(self, hf_dataset, transform=None):
        self.dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item['image']
        label = item['label']

        if self.transform:
            image = self.transform(image)

        return image, label

# Load data
train_data, test_data, train_transform, val_transform, class_names = prepare_data()

# Create datasets
train_dataset = ImageDataset(train_data, train_transform)
test_dataset = ImageDataset(test_data, val_transform)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, persistent_workers=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, persistent_workers=True)

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Classes: {class_names}")

In [ ]:
# Denormalize for visualization
mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

def denormalize(tensor):
    return tensor * std + mean

def visualize_image_samples(images, labels):
    """Visualize some samples"""
    # Create grid
    grid = make_grid(denormalize(images), nrow=4, padding=2)

    plt.figure(figsize=(5, 5))
    plt.imshow(grid.permute(1, 2, 0).clamp(0, 1))
    plt.title("Training Samples")
    plt.axis('off')

    # Print labels
    print("Labels for displayed images:")
    for i in range(16):
        print(f"{class_names[labels[i]]}", end="  ")
        if (i + 1) % 4 == 0:
            print()

    plt.show()
# Get a batch of training data
data_iter = iter(train_loader)
images, labels = next(data_iter)
visualize_image_samples(images[:16], labels[:16])

## Part 2: Understanding ViT *Architecture*

### 2.1 Image Patches: From Pixels to Tokens

In [ ]:
def extract_patches(image, patch_size=16):
    """Extract non-overlapping patches from image"""
    # image shape: (C, H, W)
    C, H, W = image.shape
    print(C,H,W)
    #TODO 1: Calculate number of patches and patches along the width (w) and heigh (h)
    num_patches_h = None # YOUR CODE HERE
    num_patches_w = None # YOUR CODE HERE
    patches = None # YOUR CODE HERE
    #ENDTODO ------------------------------------------------------------

    return patches, num_patches_h, num_patches_w

transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
image = train_data[0]['image']
image_tensor = transform(image)

patches, num_h, num_w = extract_patches(image_tensor, patch_size=32)
print(len(patches))

# Visualize
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Original image
axes[0, 0].imshow(image)
axes[0, 0].set_title("Original Image")
axes[0, 0].axis('off')

# Show first few patches
for i in range(8):
    if i == 0:
        continue
    patch_idx = i - 1
    if patch_idx < len(patches):
        patch = patches[patch_idx].permute(1, 2, 0)
        row, col = divmod(i, 4)
        axes[row, col].imshow(patch)
        axes[row, col].set_title(f"Patch {patch_idx}")
        axes[row, col].axis('off')

plt.tight_layout()
plt.show()

print(f"Image shape: {image_tensor.shape}")
print(f"Number of patches: {len(patches)}")
print(f"Patch shape: {patches[0].shape}")

### 2.2 The Model Architecture

In [ ]:
class PatchEmbedding(nn.Module):
    """Convert image patches to embeddings"""

    def __init__(self, img_size=224, patch_size=16, in_channels=3, embed_dim=768):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2

        #TODO 2: Implement patch embedding using convolution
        # Hint: Conv2d with kernel_size=patch_size, stride=patch_size
        self.projection = None # YOUR CODE HERE
        #ENDTODO ------------------------------------------------------------
        

        # Learnable position embeddings
        self.position_embeddings = nn.Parameter(
            torch.randn(1, self.num_patches + 1, embed_dim) * 0.02
        )

        # Class token (like BERT's [CLS] token)
        self.class_token = nn.Parameter(torch.randn(1, 1, embed_dim) * 0.02)

    def forward(self, x):
        batch_size = x.shape[0]

        #TODO 2: Implement forward pass
        # 1. Extract patches using convolution # Output shape: (batch_size, embed_dim, num_patches_h, num_patches_w)
        # 2. Flatten spatial dimensions  (height, width) of the patches and transpose to the standard transformer format (batch_size, seq_len, embed_dim). # (B, E, H, W) -> (B, E, N) -> (B, N, E) where N is num_patches
        # 3. Add class token. Prepend the class token to the sequence of patches. Expand the class token to match the batch size.
        # 4. Add position embeddings. This is broadcasted across the batch dimension.
        
        # Extract patches
        x =  None # YOUR CODE HERE
        x =  None # YOUR CODE HERE: flatten and transpose
        
        # Add class token
        class_tokens = None # YOUR CODE HERE
        x =  None # YOUR CODE HERE: concatenate
        
        # Add position embeddings
        x =  None # YOUR CODE HERE
        #ENDTODO ------------------------------------------------------------

        return x

In [ ]:
# The ViT Block
class ViTBlock(nn.Module):
    """Single Vision Transformer block"""

    def __init__(self, embed_dim=768, num_heads=12, mlp_ratio=4, dropout=0.1):
        super().__init__()

        # Multi-head self-attention
        self.attention = nn.MultiheadAttention(
            embed_dim, num_heads, dropout=dropout, batch_first=True
        )

        # MLP (Feed-forward)
        mlp_dim = int(embed_dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, mlp_dim),
            nn.GELU(),  # ViT uses GELU instead of ReLU
            nn.Dropout(dropout),
            nn.Linear(mlp_dim, embed_dim),
            nn.Dropout(dropout)
        )

        # Layer normalization (pre-norm like in ViT)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)

    def forward(self, x):
        #TODO 4: Implement ViT block forward pass with pre-normalization
        # Note: ViT uses pre-norm (norm before attention/mlp) unlike original transformer
        
        # Self-attention with pre-norm
        residual = x
        x = None # YOUR CODE HERE: apply norm1
        x, _ = None # YOUR CODE HERE: apply attention  (query, key, and value are the same)
        x = None # YOUR CODE HERE: add residual
        
        # MLP with pre-norm
        residual = x
        x = None # YOUR CODE HERE: apply norm2
        x = None # YOUR CODE HERE: apply mlp
        x = None # YOUR CODE HERE: add residual
        #ENDTODO ------------------------------------------------------------

        return x

In [ ]:
if not ON_COLAB:
    import otter
    from ece4078.Utility import pretty_print_otter
    grader = otter.Notebook(tests_dir = "Practical10_Support/tests")
    grader.check_all()

# [NOT GRADED] Vision Transformer in action

## 1. Putting it all together

In [ ]:
class VisionTransformer(nn.Module):
    """Complete Vision Transformer implementation"""

    def __init__(self, img_size=224, patch_size=16, in_channels=3, num_classes=1000,
                 embed_dim=768, num_layers=12, num_heads=12, mlp_ratio=4, dropout=0.1):
        super().__init__()

        self.num_classes = num_classes
        self.embed_dim = embed_dim

        # Patch embedding
        self.patch_embed = PatchEmbedding(img_size, patch_size, in_channels, embed_dim)

        # Transformer blocks
        self.blocks = nn.ModuleList([
            ViTBlock(embed_dim, num_heads, mlp_ratio, dropout)
            for _ in range(num_layers)
        ])

        # Final layer norm
        self.norm = nn.LayerNorm(embed_dim)

        # Classification head
        self.head = nn.Linear(embed_dim, num_classes)

        # Initialize weights
        self.apply(self._init_weights)

    def _init_weights(self, module):
        """Initialize weights following ViT paper"""
        if isinstance(module, nn.Linear):
            nn.init.trunc_normal_(module.weight, std=0.02)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)
        elif isinstance(module, nn.LayerNorm):
            nn.init.constant_(module.bias, 0)
            nn.init.constant_(module.weight, 1.0)

    def forward(self, x):
        # Patch embedding
        x = self.patch_embed(x)

        # Pass through transformer blocks
        for block in self.blocks:
            x = block(x)

        # Final norm
        x = self.norm(x)

        # Use class token for classification
        class_token = x[:, 0]

        # Classification
        output = self.head(class_token)

        return output

# Create model
model = VisionTransformer(
    img_size=224,
    patch_size=16,
    num_classes=len(class_names),
    embed_dim=384,   # Smaller for faster training
    num_layers=6,    # Smaller for demo
    num_heads=6
).to(device)

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

## 2. Let's try supervised classification (Standard Training)

In [ ]:
def train_supervised(model, train_loader, val_loader, num_epochs=5):
    """Standard supervised training for image classification"""

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.05)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    train_losses = []
    val_accuracies = []

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_loss = 0

        for batch_idx, (images, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}")):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Print progress every 100 batches
            if batch_idx % 100 == 0:
                print(f"Batch {batch_idx}, Loss: {loss.item():.4f}")

        # Validation phase
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        avg_loss = total_loss / len(train_loader)

        train_losses.append(avg_loss)
        val_accuracies.append(accuracy)

        scheduler.step()

        print(f"Epoch {epoch+1}: Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

    return train_losses, val_accuracies

In [ ]:
# EXAMPLE TRAINING 
print("Starting supervised training...")
sup_losses, sup_accuracies = train_supervised(model, train_loader, test_loader, num_epochs=50)

# Inference
with torch.no_grad():
    images, labels = next(iter(test_loader))
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    # visualize the first 16
    visualize_image_samples(images[:16].to('cpu'), labels[:16].to('cpu'))
    print([class_names[i] for i in predicted[:16]])